In [1]:
import pandas as pd
import numpy as np
import huggingface_hub
import torch
import transformers
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer
from sklearn.model_selection import train_test_split
import os

In [2]:
os.environ["TOKENIZERS_PARALLELISM"] = "false"
os.environ["WANDB_DISABLED"] = 'true'
#os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
#os.environ["CUDA_VISIBLE_DEVICES"]="2"

In [3]:
torch.cuda.device_count()

2

In [4]:
huggingface_hub.login()

In [5]:
f = open('10288320_anomaly_rm.txt')
data = f.readlines()
del data[0]

In [6]:
stopword = 'a, are, an, the, and, or, but, if, then, because, about, above, after, all, also, although, am, an, and, any, as, at, be, because, been, before, being, between, both, but, by, came, can, come, could, did, do, does, each, else, for, from, get, got, had, has, have, he, her, here, him, himself, his, how, i, if, in, into, is, it, its, just, like, make, many, me, might, more, most, much, must, my, never, no, nor, not, now, of, on, only, or, other, our, out, over, said, same, see, should, since, so, some, still, such, take, than, that, the, their, them, then, there, these, they, this, those, through, to, too, under, up, use, very, want, was, way, we, well, were, what, when, where, which, while, who, will, with, would, you, your'.replace(' ','').split(',')

In [7]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")

In [8]:
def preprocess_function(input_text):
    return tokenizer(input_text, truncation=True)

In [9]:
library_headline = dict()
library_short_description = dict()
classes = []
class_label_dict = dict()
label_class_dict = dict()

count=0
for i in data:
    token = i.replace('\n','').lower().split('\t')
    if(len(token) != 6):
        continue
    if token[2] not in classes:
        classes.append(token[2])
        label_class_dict[count] = token[2]
        class_label_dict[token[2]] = count
        count = count + 1

for i in classes:
    library_headline[i] = dict()
    library_short_description[i] = dict()

In [10]:
import evaluate
accuracy = evaluate.load("accuracy")

In [42]:
model = AutoModelForSequenceClassification.from_pretrained(
    "distilbert-base-uncased", num_labels=len(classes), id2label=label_class_dict, label2id=class_label_dict
).to('cuda:1')

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForSequenceClassification: ['vocab_transform.weight', 'vocab_layer_norm.bias', 'vocab_layer_norm.weight', 'vocab_transform.bias', 'vocab_projector.weight', 'vocab_projector.bias']
- This IS expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier

In [12]:
import numpy as np

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return accuracy.compute(predictions=predictions, references=labels)

In [13]:
from transformers import DataCollatorWithPadding

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [14]:
def preprocess_function(input_text):
    return tokenizer(input_text, truncation=True)

In [15]:
t,e = train_test_split(data, test_size=0.3)

In [16]:
class myDataset(torch.utils.data.Dataset):
    def __init__(self):
        #self.data = data[:int(len(data)*0.7)]
        self.data = t
    def __len__(self):
        return len(self.data)
    def __getitem__(self, idx):
        token = self.data[idx].replace('\n','').lower().split('\t')
        x = preprocess_function(token[1]+' '+token[3])
        y = class_label_dict[token[2]]
        return {'text':token[1]+' '+token[3], 'label':y, 'input_ids':x['input_ids'], 'attention_mask':x['attention_mask']}
    #dict_keys(['text', 'label', 'input_ids', 'attention_mask'])

In [17]:
class myDataset_eval(torch.utils.data.Dataset):
    def __init__(self):
        #self.data = data[int(len(data)*0.7):]
        self.data = e
    def __len__(self):
        return len(self.data)
    def __getitem__(self, idx):
        token = self.data[idx].replace('\n','').lower().split('\t')
        x = preprocess_function(token[1]+' '+token[3])
        y = class_label_dict[token[2]]
        return {'text':token[1]+' '+token[3], 'label':y, 'input_ids':x['input_ids'], 'attention_mask':x['attention_mask']}

In [18]:
train_dataset = myDataset()
eval_dataset = myDataset_eval()

In [19]:
training_args = TrainingArguments(
    output_dir="my_awesome_model",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=2,
    weight_decay=0.01,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    push_to_hub=True,
)

trainer = Trainer(
    model=model.to('cuda:1'),
    args=training_args,
    train_dataset=train_dataset,
    #train_dataset=tokenized_imdb['train'],
    eval_dataset=eval_dataset,
    #eval_dataset=tokenized_imdb['test'],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

#trainer.train()

Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).
/home/gkook/school/nlp/doc_classifier/my_awesome_model is already a clone of https://huggingface.co/gkook/my_awesome_model. Make sure you pull the latest changes with `repo.git_pull()`.


In [21]:
trainer.train()

Epoch,Training Loss,Validation Loss


TrainOutput(global_step=9372, training_loss=0.8877203400400628, metrics={'train_runtime': 1196.5697, 'train_samples_per_second': 250.588, 'train_steps_per_second': 7.832, 'total_flos': 5946549397219428.0, 'train_loss': 0.8877203400400628, 'epoch': 2.0})

In [43]:
model.load_state_dict(torch.load('/home/gkook/school/nlp/doc_classifier/my_awesome_model/checkpoint-9372/pytorch_model.bin'))

<All keys matched successfully>

In [65]:
model.config.id2label[model(**tokenizer(e[0].lower().split('\t')[1]+' '+e[0].lower().split('\t')[3], return_tensors='pt').to('cuda:1')).logits.argmax().item()]

'politics'

In [69]:
trainer.evaluate()

/home/gkook/.local/lib/python3.9/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


{'eval_loss': 0.8290209770202637,
 'eval_accuracy': 0.7499260734907319,
 'eval_runtime': 105.5734,
 'eval_samples_per_second': 608.61,
 'eval_steps_per_second': 19.02,
 'epoch': 2.0}